# Data Collection

This assumes that you have registered an account with Kaggle, and have created a json key to interact with Kaggle's API.  

In [110]:
import os
import shutil
import pandas as pd

In [111]:
os.chdir(os.path.expanduser('~'))
os.chdir('Desktop/skin')

current_dir = os.getcwd()
current_dir

'/home/pxp/Desktop/skin'

In [112]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

In [113]:
meta_data = pd.read_csv("inputs/cancer-dataset/HAM10000_metadata.csv")
meta_data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


# Data Preparation

## Data Cleaning
Check and remove non-image files

In [114]:
my_data_dir='inputs/cancer-dataset/photos'

## Split train validation test set

In [115]:
meta_data = pd.read_csv("inputs/cancer-dataset/HAM10000_metadata.csv")
meta_data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [116]:
image_dir_path = 'inputs/cancer-dataset/photos' 
train_dir = 'inputs/cancer-dataset/train'
test_dir = 'inputs/cancer-dataset/test'
val_dir = 'inputs/cancer-dataset/validate'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

cancer_types = meta_data["dx"].unique()
for cancer_type in cancer_types:
    os.makedirs(os.path.join(train_dir, cancer_type), exist_ok=True)
    os.makedirs(os.path.join(test_dir, cancer_type), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cancer_type), exist_ok=True)


In [117]:
meta_data = meta_data.sample(frac=1).reset_index(drop=True)

test_count = int(meta_data.shape[0]*.7)
val_count = int(meta_data.shape[0]*.2)

for i in range(meta_data.shape[0]):
    image_path = os.path.join(image_dir_path, meta_data["image_id"][i]+".jpg")
    if i < test_count:
        shutil.copy(image_path, os.path.join(train_dir, meta_data["dx"][i]))
    elif i < test_count + val_count:
        shutil.copy(image_path, os.path.join(test_dir, meta_data["dx"][i]))
    else:
        shutil.copy(image_path, os.path.join(val_dir, meta_data["dx"][i]))

In [128]:
# count files in the train directory
# print('Train:')
# print('----------------------')
# for cancer_type in cancer_types:
#     print(cancer_type, len(os.listdir(os.path.join(train_dir, cancer_type))))
# print('\n')

# print('Test:')
# print('----------------------')
# for cancer_type in cancer_types:
#     print(cancer_type, len(os.listdir(os.path.join(test_dir, cancer_type))))
# print('\n')

# print('Validate:')
# print('----------------------')
# for cancer_type in cancer_types:
#     print(cancer_type, len(os.listdir(os.path.join(val_dir, cancer_type))))

paths = [train_dir, test_dir, val_dir]
names = ['train', 'test', 'validate']
for i in range(3):
    print(names[i])
    print('----------------------')
    for cancer_type in cancer_types:
        print(cancer_type, len(os.listdir(os.path.join(paths[i], cancer_type))))
    print('\n')

train
----------------------
bkl 759
nv 4720
df 82
mel 763
vasc 104
bcc 360
akiec 222


test
----------------------
bkl 242
nv 1326
df 20
mel 206
vasc 23
bcc 105
akiec 81


validate
----------------------
bkl 98
nv 659
df 13
mel 144
vasc 15
bcc 49
akiec 24




Data is ready for training.